# Deep Crazyhouse


## Conversion of the Matrix- to Board Representation

* file: mat_to_board_demo.ipynb
* brief: Loads in a png-file from the lichess crazyhouse dataset and converts it to plane representation. The plane representations can later be used by a convolutional neural network.

* author: QueensGambit
* contact: johannes.czech@stud.tu-darmstadt.de
* version: 2018-07-01 visualized first game

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.insert(0,'../../../')
from DeepCrazyhouse.src.preprocessing.load_pgn_parallel import convert_pgn_to_planes
from DeepCrazyhouse.src.preprocessing.plane_converter import get_mat_of_current_board
from DeepCrazyhouse.src.model.crazy_house.move_presentation import get_move_representation
from DeepCrazyhouse.src.preprocessing.util import *
import logging

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import datetime
from time import time
import chess
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
t_s = time()
s_idcs, x, yv, yp, pgn_datasets = load_pgn_dataset('../config.json')
t_e = time() - t_s
print('Elapsed time for loading(hh:mm:ss): ' + str(datetime.timedelta(seconds=round(time() - t_s))))

In [ ]:
# Unflatten the x_train vector for these test
pgn_dataset = pgn_datasets[0]
s_idcs = s_idcs[0]
x = x[0]
yv = yv[0]
yp = yp[0]

### File Structure of the Dataset

In [ ]:
pgn_dataset.tree()

### Show the Metadata of the first Game

In [ ]:
metadata = np.array(pgn_dataset['metadata'])
metadata[0, :]
metadata[1, :]

### Game URL

In [ ]:
print('The game can also be found at:', metadata[1, 1])

### Visualize the first Game Moves

In [ ]:
def show_board_pos(x, yp):
    
    board = get_mat_of_current_board(x) 
    print("{0}'s turn".format(chess.COLOR_NAMES[board.turn]))
    print("black/white: {0}".format(board.pockets))
    
    planes = yp.reshape(78, 8, 8)
    next_move = get_move_representation(board, planes, is_white_to_move=board.turn)
    
    print("next move: {0}".format(next_move))

    return board, next_move

In [ ]:
def set_chess_board():
    ax = plt.gca()
    ax.set_xticks([0,1,2,3,4,5,6,7])
    ax.set_xticklabels( ('a','b','c','d','e','f','g','h')) #, color=my_cmap)

    ax.set_yticks([0,1,2,3,4,5,6,7])
    ax.set_yticklabels(range(1,9)[::-1],color='black')

    ax.tick_params(labeltop=True, labelright=True)


In [ ]:
# https://stackoverflow.com/questions/12073306/customize-colorbar-in-matplotlib
#255 143 31
cdict = {'red':  ( (0.0, 0.0, 0.0),   # <- at 0.0, the red component is 0
                   (0.5, 255/255, 255/255),   # <- at 0.5, the red component is 1
                   (1.0, 1.0, 1.0)),  # <- at 1.0, the red component is 0

         'green': ((0.0, 0.0, 0.0),   # <- etc.
                   (0.5, 143/255, 143/255),
                   (1.0, 1.0, 1.0)),

         'blue':  ((0.0, 0.0, 0.0),
                   (0.5, 31/255, 31/255),
                   (1.0, 1.0, 1.0))
         }

my_cmap = matplotlib.colors.LinearSegmentedColormap('my_colormap',cdict,256)

#for yp_cur in yp[:10]:
yp_mat = yp[2].reshape((78, 8, 8))
for i in range(78):
    tmp_sum = np.sum(yp_mat[i])
    #yp_mat[60, 0, 6] = 1.
    if tmp_sum != 0:
        print('Movement Channel:',i)
        #yp_mat[i, :, :] = 0
        #yp_mat[60, 0, 6] = 1.
        break
        
#plt.imshow(np.flipud(yp_mat[i]))


#for i in range(64):
#    if i % 2 == 0:
#        row, col = get_row_col(i)
#        chess_pattern[row, col] = 1

chess_pattern_flat = np.zeros(9*9)
chess_pattern_flat[::2] = 1


chess_pattern = chess_pattern_flat.reshape((9,9))[:8,:8]

# highlight the move
sel_idx = np.flipud(yp_mat[i]).argmax()

r, c = get_row_col(sel_idx)
chess_pattern[r, c] = 0.5

plt.imshow(chess_pattern, cmap=my_cmap)
set_chess_board()



plt.show()

In [ ]:
b, move = show_board_pos(x[i], yp[i])

In [ ]:
b.

In [ ]:
b(move.to_square)

In [ ]:
for i in range(17):
    show_board_pos(x[i], yp[i])

### Visualize the last 10 moves

In [ ]:
for i in range(s_idcs[1]-10, s_idcs[1]):
    show_board_pos(x[i], yp[i])

### Show Black's Pawn Formation in the last Position

In [ ]:
plt.imshow(x[s_idcs[1]-1][0], cmap='binary')